In [153]:
import pandas as pd
from datetime import datetime
from functools import reduce


In [154]:
df_business_Confidence = pd.read_csv('../input/Business_Confidence_Index.csv') 
df_consumer_Confidence = pd.read_csv('../input/Consumer_Confidence_Index.csv') 
df_inflation = pd.read_csv('../input/Inflation.csv') 
df_vola = pd.read_csv('../input/BTC_Volatility.csv')

In [155]:
def get_var_name(variable):
     for name, value in globals().items():
        if value is variable:
            return name

def clean_df(*dfs):
    cleaned_dfs = []
    for i,df in enumerate(dfs):
        name = get_var_name(dfs[i])[3:].title()
        df = df.loc[df['LOCATION']=='G-20'].reset_index(drop=True)[['TIME', 'Value']]
        df.rename(columns={"Value":f'{name}'}, inplace=True)
        df['TIME'] = [datetime.strptime(j,"%Y-%m") for j in df['TIME']]
        df.set_index('TIME', inplace=True)
        cleaned_dfs.append(df)
    return cleaned_dfs 

df_bci, df_cci,df_inflation = clean_df(df_business_Confidence, df_consumer_Confidence, df_inflation)

In [156]:
# drop redundant cols
df_vola.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)

#convert Date to datetime format
df_vola['DateTime'] = pd.to_datetime(df_vola['DateTime'], format='%d/%m/%Y %H:%M')
df_vola.rename(columns={'30-Day BTC/USD Volatility':'BTC/USD_Volatility_Index'}, inplace=True)
df_vola.set_index('DateTime', inplace=True)

In [157]:
dfs = [df_bci, df_cci,df_inflation, df_vola]

df_merged = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True,
                                            how='outer'), dfs)



In [160]:
for i in df_merged.columns[0:3]:
    df_merged[i] = df_merged[i].fillna(method='ffill')
        
df_merged

/tmp/ipykernel_14384/2326111514.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged[i] = df_merged[i].fillna(method='ffill')


,Business_Confidence,Consumer_Confidence,Inflation,BTC/USD_Volatility_Index
2010-08-16,NaN,NaN,NaN,12.056917
2010-08-17,NaN,NaN,NaN,12.057903
2010-08-18,NaN,NaN,NaN,12.096542
2010-08-19,NaN,NaN,NaN,12.023751
2010-08-20,NaN,NaN,NaN,11.967747
...,...,...,...,...
2024-01-20,99.28704,96.72697,5.783826,2.012616
2024-01-21,99.28704,96.72697,5.783826,2.020418
2024-01-22,99.28704,96.72697,5.783826,2.021783
2024-01-23,99.28704,96.72697,5.783826,2.062327


In [161]:

df_merged.to_csv('../output/df_merged.csv')